In [24]:
import requests  # 引入 requests 模組，用於發送 HTTP 請求
from bs4 import BeautifulSoup  # 引入 BeautifulSoup 模組，用於解析 HTML
import pandas as pd  # 引入 pandas 模組，用於資料處理

# 定義抓取天氣資料的函數
def scrape_weather_data():
    # 發送 GET 請求以取得網頁內容
    resp = requests.get("https://www.cwa.gov.tw/V8/C/W/Observe/MOD/24hr/46692.html")
    # 使用 BeautifulSoup 解析 HTML 內容
    soup = BeautifulSoup(resp.text, 'html.parser')

    data = []  # 初始化一個空列表，用於存儲抓取的資料
    # 找到所有的 <tr> 標籤
    for row in soup.find_all("tr"):
        # 取得每行中的所有字串並去除多餘空白
        row_data = list(row.stripped_strings)
        # 如果這行有 14 個元素，表示是我們需要的資料
        if len(row_data) == 14:
            data.append([
                row_data[0],  # 日期
                row_data[1],  # 時間
                row_data[2],  # 溫度
                row_data[3],  # 濕度
                row_data[4],  # 風向
                row_data[5],  # 風速
                row_data[6],  # 陣風
                row_data[10], # 能見度
                row_data[11], # 海平面氣壓
            ])

    # 定義欄位名稱
    columns = ["日期", "時間", "溫度", "濕度", "風向", "風速", "陣風", "能見度", "海平面氣壓"]
    # 使用 pandas 將資料轉換為 DataFrame
    df = pd.DataFrame(data, columns=columns)
    print(df)  # 打印 DataFrame
    return df  # 返回 DataFrame

if __name__ == "__main__":
    df = scrape_weather_data()  # 呼叫 scrape_weather_data 函數
    # 將 DataFrame 保存為 CSV 文件，指定編碼為 utf-8-sig
    df.to_csv("weather_data.csv", index=False, encoding="utf-8-sig")
    print("天氣資料已儲存至 weather_data.csv")


        日期     時間    溫度  濕度   風向   風速 陣風 能見度   海平面氣壓
0    07/03  00:30  27.2  81    南  1.9  2  84  1011.7
1    07/03  00:20  27.0  81    南  1.4  1  85  1011.7
2    07/03  00:10  27.2  81    南  1.5  1  84  1011.7
3    07/03  00:00  27.0  81    南  1.6  2  85  1011.8
4    07/02  23:50  27.0  81    南  1.5  1  86  1011.8
..     ...    ...   ...  ..  ...  ... ..  ..     ...
144  07/02  00:20  28.1  83  西南西  1.2  1  96  1007.1
145  07/02  00:10  28.4  83  南南西  1.1  1  94  1007.2
146  07/02  00:00  28.1  83   東南  0.9  1  95  1007.0
147  07/01  23:50  28.1  83   東南  1.1  1  95  1007.1
148  07/01  23:40  28.3  83  南南東  1.0  1  93  1007.5

[149 rows x 9 columns]
天氣資料已儲存至 weather_data.csv


In [26]:
import pandas as pd  # 引入 pandas 模組，用於資料處理
import pyodbc  # 引入 pyodbc 模組，用於與資料庫連接

# 定義清理資料的函數
def clean_data(df):
    # 將無效的值替換為 NaN，並轉換資料型別
    df['溫度'] = pd.to_numeric(df['溫度'], errors='coerce')
    df['濕度'] = pd.to_numeric(df['濕度'], errors='coerce')
    df['風速'] = pd.to_numeric(df['風速'], errors='coerce')
    df['陣風'] = pd.to_numeric(df['陣風'], errors='coerce')
    df['能見度'] = pd.to_numeric(df['能見度'], errors='coerce')
    df['海平面氣壓'] = pd.to_numeric(df['海平面氣壓'], errors='coerce')
    df = df.dropna()  # 移除包含 NaN 的行
    return df  # 返回清理後的 DataFrame

# 定義將資料插入資料庫的函數
def save_to_database(df):
    # 設定資料庫連接字串
    conn_str = (
        "DRIVER={ODBC Driver 17 for SQL Server}; "
        "SERVER=WEIWEI; "
        "DATABASE=model; "
        "UID=wei; "
        "PWD=132476; "
        "TrustServerCertificate=yes;"
    )
    conn = pyodbc.connect(conn_str)  # 連接資料庫
    cursor = conn.cursor()  # 獲取游標

    # 創建資料表（如果不存在）
    cursor.execute("""
    IF OBJECT_ID('weather_data', 'U') IS NULL
    CREATE TABLE weather_data (
        日期 VARCHAR(10),
        時間 VARCHAR(5),
        溫度 FLOAT,
        濕度 INT,
        風向 VARCHAR(10),
        風速 FLOAT,
        陣風 FLOAT,
        能見度 FLOAT,
        海平面氣壓 FLOAT
    )
    """)

    # 插入資料
    for index, row in df.iterrows():
        cursor.execute("""
        INSERT INTO weather_data (日期, 時間, 溫度, 濕度, 風向, 風速, 陣風, 能見度, 海平面氣壓)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, row['日期'], row['時間'], row['溫度'], row['濕度'], row['風向'], row['風速'], row['陣風'], row['能見度'], row['海平面氣壓'])
        print(f"資料插入成功: {row['日期']} {row['時間']}")

    conn.commit()  # 提交變更

    # 查詢確認資料數量
    cursor.execute("SELECT COUNT(*) FROM weather_data")
    count = cursor.fetchone()[0]
    print(f"資料庫中共有 {count} 筆資料")

    cursor.close()  # 關閉游標
    conn.close()  # 關閉連接

if __name__ == "__main__":
    df = pd.read_csv("weather_data.csv")  # 從 CSV 文件讀取 DataFrame
    df = clean_data(df)  # 清理資料
    save_to_database(df)  # 呼叫 save_to_database 函數

資料插入成功: 07/03 00:30
資料插入成功: 07/03 00:20
資料插入成功: 07/03 00:10
資料插入成功: 07/03 00:00
資料插入成功: 07/02 23:50
資料插入成功: 07/02 23:40
資料插入成功: 07/02 23:30
資料插入成功: 07/02 23:20
資料插入成功: 07/02 23:10
資料插入成功: 07/02 23:00
資料插入成功: 07/02 22:50
資料插入成功: 07/02 22:40
資料插入成功: 07/02 22:30
資料插入成功: 07/02 22:20
資料插入成功: 07/02 22:10
資料插入成功: 07/02 22:00
資料插入成功: 07/02 21:50
資料插入成功: 07/02 21:40
資料插入成功: 07/02 21:30
資料插入成功: 07/02 21:20
資料插入成功: 07/02 21:10
資料插入成功: 07/02 21:00
資料插入成功: 07/02 20:50
資料插入成功: 07/02 20:40
資料插入成功: 07/02 20:30
資料插入成功: 07/02 20:20
資料插入成功: 07/02 20:10
資料插入成功: 07/02 19:40
資料插入成功: 07/02 19:30
資料插入成功: 07/02 19:20
資料插入成功: 07/02 19:10
資料插入成功: 07/02 19:00
資料插入成功: 07/02 18:50
資料插入成功: 07/02 18:40
資料插入成功: 07/02 18:30
資料插入成功: 07/02 18:20
資料插入成功: 07/02 18:10
資料插入成功: 07/02 18:00
資料插入成功: 07/02 17:50
資料插入成功: 07/02 17:40
資料插入成功: 07/02 17:30
資料插入成功: 07/02 17:20
資料插入成功: 07/02 17:10
資料插入成功: 07/02 17:00
資料插入成功: 07/02 16:50
資料插入成功: 07/02 16:40
資料插入成功: 07/02 16:30
資料插入成功: 07/02 16:20
資料插入成功: 07/02 16:10
資料插入成功: 07/02 16:00


In [28]:
import pandas as pd  # 引入 pandas 模組，用於資料處理
import pyodbc  # 引入 pyodbc 模組，用於與資料庫連接

# 定義從資料庫取出資料並保存為 Excel 文件的函數
def export_to_excel():
    # 設定資料庫連接字串
    conn_str = (
        "DRIVER={ODBC Driver 17 for SQL Server}; "
        "SERVER=WEIWEI; "
        "DATABASE=model; "
        "UID=wei; "
        "PWD=132476; "
        "TrustServerCertificate=yes;"
    )
    conn = pyodbc.connect(conn_str)  # 連接資料庫
    query = "SELECT * FROM weather_data"  # 定義 SQL 查詢
    df = pd.read_sql(query, conn)  # 執行查詢並讀取結果到 DataFrame
    conn.close()  # 關閉連接

    # 將 DataFrame 保存為 Excel 文件
    desktop_path = "C:/Users/Willi/OneDrive/Desktop/天氣2/weather_data.xlsx"
    df.to_excel(desktop_path, index=False)
    print(f"資料已保存至 {desktop_path}")

if __name__ == "__main__":
    export_to_excel()  # 呼叫 export_to_excel 函數


資料已保存至 C:/Users/Willi/OneDrive/Desktop/天氣2/weather_data.xlsx


C:\Users\Willi\AppData\Local\Temp\ipykernel_20748\137953713.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)  # 執行查詢並讀取結果到 DataFrame


In [ ]:
from flask import Flask, render_template  # 引入 Flask 模組，用於創建網頁應用
import pandas as pd  # 引入 pandas 模組，用於資料處理
import plotly.express as px  # 引入 plotly.express 模組，用於創建圖表

app = Flask(__name__)  # 創建 Flask 應用

@app.route('/')
def show_excel():
    # 讀取 Excel 文件
    desktop_path = "C:/Users/Willi/OneDrive/Desktop/天氣2/weather_data.xlsx"
    df = pd.read_excel(desktop_path)

    # 將 DataFrame 轉換為 HTML 表格
    html_table = df.to_html(index=False)

    # 創建長條圖
    fig = px.bar(df, x='日期', y='溫度', title='每日溫度變化')
    graph_html = fig.to_html(full_html=False)

    # 渲染模板並返回 HTML
    return render_template('weather_data.html', table=html_table, graph=graph_html)

if __name__ == "__main__":
    app.run()  # 啟動 Flask 應用